# Plot of RHS in $L_1$ error between $\mathcal{L}_{t}$ and $\mathcal{L}_{t}^{PCA}$

For any $p \geq 1$, plot of

$$ \frac{1}{n} \sum_{i=1}^{n} \Lambda^{i} \, \left(\frac{\sum_{k=3}^{n} \nu^{k} \times (u^{i, k})^{2}}{\sum_{k=1}^{n} \nu^{k} \times (u^{i, k})^{2}}\right)^{\frac{p}{2(p+1)}}. $$

Define also $C_{p} = \frac{9}{\sqrt{\pi}} \Gamma(\frac{1+p}{2})^{\frac{1}{p+1}}$. To simplify, we take $\Lambda^i = 1$ for all $i$.


In [ ]:
import numpy as np
from tqdm import tqdm
import matplotlib.pyplot as plt

In [ ]:
def rhs_func(n, p, b_min, b_max, t=1.0):
    b = np.random.uniform(b_min, b_max, size=n)
    b_rep = np.tile(b, (n, 1))
    rho = np.random.uniform(-1, 1, size=n)
    rho_rep = np.tile(rho, (n, 1))
    cov_X = (
        (rho_rep * rho_rep.T)
        * (1 - np.exp(-(b_rep + b_rep.T) * t))
        / (b_rep + b_rep.T)
    )
    U, eig_X, U_T = np.linalg.svd(cov_X)
    num = np.sum(eig_X[2:, None] * pow(U[2:, :], 2), axis=0)
    denom = np.sum(eig_X[:, None] * pow(U[:, :], 2), axis=0)
    res = np.sum(pow(num / denom, 0.5 * p / (p + 1)))
    return res / n


def mc_rhs_func(tab_n, n_mc, p, b_min, b_max):
    bound = np.zeros(tab_n.shape[0])
    bound_err = np.zeros(tab_n.shape[0])

    for idx in tqdm(range(tab_n.shape[0])):
        list = []
        for _ in range(n_mc):
            _rhs = rhs_func(n=tab_n[idx], p=p, b_min=b_min, b_max=b_max)
            list.append(_rhs)
        bound[idx] = np.array(list).mean()
        bound_err[idx] = 1.96 * np.array(list).std() / np.sqrt(n_mc)
    return bound, bound_err

In [ ]:
n_mc = 30
tab_n = np.arange(100, 1500, 100)

In [ ]:
bound_01, bound_err_01 = mc_rhs_func(
    tab_n=tab_n, n_mc=n_mc, p=1, b_min=0, b_max=1
)
bound_14, bound_err_14 = mc_rhs_func(
    tab_n=tab_n, n_mc=n_mc, p=1, b_min=1, b_max=4
)
bound_010, bound_err_010 = mc_rhs_func(
    tab_n=tab_n, n_mc=n_mc, p=1, b_min=0, b_max=10
)

In [ ]:
MARKERSIZE = 10
ELINEWIDTH = 1
CAPSIZE = 3

In [ ]:
fig, ax = plt.subplots()
ax.errorbar(
    tab_n,
    bound_01,
    yerr=bound_err_01,
    markersize=MARKERSIZE,
    elinewidth=ELINEWIDTH,
    fmt="*",
    capsize=CAPSIZE,
    label="$b=$ Uniform [0, 1]",
)
ax.errorbar(
    tab_n,
    bound_14,
    yerr=bound_err_14,
    markersize=MARKERSIZE,
    elinewidth=ELINEWIDTH,
    fmt=".",
    capsize=CAPSIZE,
    label="$b=$ Uniform [1, 4]",
)
ax.errorbar(
    tab_n,
    bound_010,
    yerr=bound_err_010,
    markersize=MARKERSIZE,
    elinewidth=ELINEWIDTH,
    fmt="x",
    capsize=CAPSIZE,
    label="$b=$ Uniform [0, 10]",
)
ax.set_xlabel("$n$")
ax.legend()
plt.show()